In [2]:
!pip install xlsxwriter


Der Befehl "version" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.


In [4]:
#import all kinds of stuff

import tweepy
import pandas as pd
import webbrowser
import matplotlib.pyplot as plt
import datetime
from textblob import TextBlob
#from google.colab import files
import xlsxwriter

#set the initials

api_key = "F8OtWY5GuXRhiMBFld7JyEWEg"
api_secret_key = "b4natkFo1GtTlComJ52AXUbLAW7gUkVGaes2dbgRtiV6NZ2g8W"
access_token = "1471177476730769432-Sfk7enNXCofv9CUEitO2Tdonjx1fjK"
access_token_secret = "wUg0W1D3mns0LkhVpGPnZOe1kj7tU67HmUiDCZyyQWBPg"
bearer_token = "AAAAAAAAAAAAAAAAAAAAABwyYQEAAAAAP125dZoGNBPLZofrLDouD6dZb4A%3DadQzf2pnGwAWsBqiuBv2vc9HblrBTzIX7fbfz2tZlU7yWrIPKL"

auth = tweepy.OAuthHandler(api_key,api_secret_key)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth)
print("Initials set")

#clean text from unneccessary words

def cleanTxt(text):
  text = re.sub(r'@[A-Za-z0-9]+' , '' , text)
  text = re.sub(r'#' , '' , text)
  text = re.sub(r'RT[\s]+' , '' , text)
  text = re.sub(r'https?:\/\/\S+' , '' , text)
  return text


#analyzing polarization
def sentiment(tweets):
  pos = 0
  neg = 0
  neu = 0

  for tw in tweets:
    analysis = TextBlob(tw)
    tweet_polarity = analysis.polarity
    if tweet_polarity > 0:
      pos = pos + 1
    elif tweet_polarity < 0:
      neg = neg + 1
    else:
      neu = neu + 1

  return [pos,neg,neu]

def sentiment_one(tweet):
  analysis = TextBlob(tweet)
  tweet_polarity = analysis.polarity
  if tweet_polarity > 0:
    return 1
  elif tweet_polarity < 0:
    return -1
  else:
    return 0

#loading data from 01.21 to 01.27
dates_array = []
day = range(21,28)
for i in range(len(day)):
  dates = datetime.datetime(2022,1,day[i])
  dates_array.append(dates.strftime("%Y/%m/%d"))

all_pos = [25,25,16,44,43,40,37]
all_neg = [11,16,14,21,15,23,12]
all_neu = [45,69,87,135,132,111,81]

old_sent_df = pd.DataFrame({'Date':dates_array , 'Number of positive':all_pos , 'Number of negative':all_neg , 'Number of neutral':all_neu})
old_sent_df

Initials set


,Date,Number of positive,Number of negative,Number of neutral
0,2022/01/21,25,11,45
1,2022/01/22,25,16,69
2,2022/01/23,16,14,87
3,2022/01/24,44,21,135
4,2022/01/25,43,15,132
5,2022/01/26,40,23,111
6,2022/01/27,37,12,81


In [5]:
#ukraine tweet pull
def pull_tweets(date_pull , query):
  num_tweets = 300
  tweets = []
  likes = []
  time = []
  sent = []
  #since_date = "2022-01-22"
  #until_date = '2022-01-22T02:00:00Z'

  #since_date = datetime.datetime(2022,1,21)
  #until_date = datetime.datetime(2022,1,22)


  since_date = date_pull
  until_date = date_pull + datetime.timedelta(days=1)

  #until_date = datetime.datetime(2022, 1, 26, 13, 00, 00)
  #q_ukraine = 'Ukraine OR Russia -is:nullcast lang:en -filter:retweets'
  #q_ukraine = "ukraine"
  q_ukraine = query


  cursor_ukraine = tweepy.Cursor(api.search, q=q_ukraine , lange='en' ,since=since_date, until=until_date,count=100000).items(num_tweets)
  #cursor_ukraine = tweepy.Paginator(api.search, q=q_ukraine , lange='en' ,since=since_date, until=until_date,count=100000).items(num_tweets)



  for i in cursor_ukraine:
    tweets.append(cleanTxt(i.text))
    likes.append(i.favorite_count)
    time.append(i.created_at)
    sent.append(sentiment_one(cleanTxt(i.text)))
    
    

  df=pd.DataFrame({'Tweets':tweets,'Likes':likes,'Time':time , 'Sentiment':sent})
  df=df.sort_values(by=['Time'])
  df.index=df['Time']
  df=df.drop('Time' ,1)

  

  

  



  return df

def save_data(dataframe , date):

  df = dataframe
  df1=df.between_time('0:00' , '3:59')
  df2=df.between_time('4:00' , '7:59' )
  df3=df.between_time('8:00' , '11:59')
  df4=df.between_time('12:00' , '15:59')
  df5=df.between_time('16:00' , '19:59')
  df6=df.between_time('20:00' , '23:59')
  
  file_name = 'tweets_pulled_from_' + date.strftime("%Y_%m_%d") + '.xlsx'
  writer = pd.ExcelWriter(file_name, engine='xlsxwriter')

  df1.to_excel(writer, sheet_name='0 - 4')
  df2.to_excel(writer, sheet_name='4 - 8')
  df3.to_excel(writer, sheet_name='8 - 12')
  df4.to_excel(writer, sheet_name='12 - 16')
  df5.to_excel(writer, sheet_name='16 - 20')
  df6.to_excel(writer, sheet_name='20 - 24')

  writer.save()
  files.download(file_name)


In [101]:
#test cell

q_ukraine = '(ukraine AND NATO) OR (ukraine AND Russia) OR (ukraine AND war) OR (russia AND War) OR (Russia AND gas) OR (war AND europe) OR (ukraine AND war) OR Putin OR (#ukraine AND #russia) lang:en -filter:retweets'

#q_ukraine = 'ukraine AND russia lang:en -filter:retweets'

full_df = []
dates_array = []

dates = datetime.datetime(2022,1,28)
dates_array.append(dates.strftime("%Y/%m/%d"))
df01_28 = pull_tweets(dates,q_ukraine)
full_df.append(df01_28)

dates = datetime.datetime(2022,1,29)
dates_array.append(dates.strftime("%Y/%m/%d"))
df01_29 = pull_tweets(dates,q_ukraine)
full_df.append(df01_29)

dates = datetime.datetime(2022,1,30)
dates_array.append(dates.strftime("%Y/%m/%d"))
df01_30 = pull_tweets(dates,q_ukraine)
full_df.append(df01_30)

dates = datetime.datetime(2022,1,31)
dates_array.append(dates.strftime("%Y/%m/%d"))
df01_31 = pull_tweets(dates,q_ukraine)
full_df.append(df01_31)

dates = datetime.datetime(2022,2,1)
dates_array.append(dates.strftime("%Y/%m/%d"))
df02_01 = pull_tweets(dates,q_ukraine)
full_df.append(df02_01)

dates = datetime.datetime(2022,2,2)
dates_array.append(dates.strftime("%Y/%m/%d"))
df02_02 = pull_tweets(dates,q_ukraine)
full_df.append(df02_02)

dates = datetime.datetime(2022,2,3)
dates_array.append(dates.strftime("%Y/%m/%d"))
df02_03 = pull_tweets(dates,q_ukraine)
full_df.append(df02_03)

dates = datetime.datetime(2022,2,4)
dates_array.append(dates.strftime("%Y/%m/%d"))
df02_04 = pull_tweets(dates,q_ukraine)
full_df.append(df02_04)

dates = datetime.datetime(2022,2,5)
dates_array.append(dates.strftime("%Y/%m/%d"))
df02_05 = pull_tweets(dates,q_ukraine)
full_df.append(df02_05)





full_df[2]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


,Tweets,Likes,Sentiment
Time,,,
2022-01-30 00:08:08,"'Russia, perhaps, will not dare': Meet the Ukr...",0,0
2022-01-30 03:15:23,Explained | Can a Russia Ukraine conflict be a...,0,1
2022-01-30 05:00:00,Russia insists it won't start a war as Ukraine...,72,-1
2022-01-30 05:18:48,Putin's miscalculations after invading Ukrain...,2,0
2022-01-30 05:32:10,Perspective | Putin’s case for invading Ukrain...,0,0
2022-01-30 07:46:20,"Explained: Russia has mobilised some 1,00,000 ...",125,0
2022-01-30 09:14:38,For Jan 29: Ukrainian military report another ...,19,1
2022-01-30 09:40:53,"Russias goal since the early 00's is ""To get...",1,1
2022-01-30 10:33:55,Freeze every account at 06:00 on Monday and m...,1,0


In [102]:
#counting the sentiment for each day
all_pos = []
all_neg = []
all_neu = []
for index in range(len(full_df)):
  all_pos.append(full_df[index]['Sentiment'].value_counts()[1])
  all_neg.append(full_df[index]['Sentiment'].value_counts()[-1])
  all_neu.append(full_df[index]['Sentiment'].value_counts()[0])


sent_df = pd.DataFrame({'Date':dates_array , 'Number of positive':all_pos , 'Number of negative':all_neg , 'Number of neutral':all_neu})
sent_df



,Date,Number of positive,Number of negative,Number of neutral
0,2022/01/28,18,9,29
1,2022/01/29,9,7,23
2,2022/01/30,9,5,29
3,2022/01/31,11,5,25
4,2022/02/01,16,12,25
5,2022/02/02,16,7,37
6,2022/02/03,12,6,31
7,2022/02/04,11,13,37
8,2022/02/05,10,6,24


In [103]:
file_name = 'tweets_between_0128_to_0205.xlsx'
writer = pd.ExcelWriter(file_name, engine='xlsxwriter')

df01_28.to_excel(writer, sheet_name='01.28')
df01_29.to_excel(writer, sheet_name='01.29')
df01_30.to_excel(writer, sheet_name='01.30')
df01_31.to_excel(writer, sheet_name='01.31')
df02_01.to_excel(writer, sheet_name='02.01')
df02_02.to_excel(writer, sheet_name='02.02')
df02_03.to_excel(writer, sheet_name='02.03')
df02_04.to_excel(writer, sheet_name='02.04')
df02_05.to_excel(writer, sheet_name='02.05')
sent_df.to_excel(writer, sheet_name='Sentiment per day (01.28-02.05)')
old_sent_df.to_excel(writer, sheet_name='Sentiment per day (01.21-01.27)')



writer.save()
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>